# Cirq Weber-QVM readout errors calibration data analysis

**Ohad Lev**

**October 2024**

---------

## Initializing desired data and containers

In [1]:
import cirq
import cirq_google


QUBIT_ORDER_12_QUBITS = [
    cirq.GridQubit(3, 3),
    cirq.GridQubit(3, 4),
    cirq.GridQubit(3, 5),
    cirq.GridQubit(3, 6),
    cirq.GridQubit(4, 3),
    cirq.GridQubit(4, 4),
    cirq.GridQubit(4, 5),
    cirq.GridQubit(4, 6),
    cirq.GridQubit(5, 3),
    cirq.GridQubit(5, 4),
    cirq.GridQubit(5, 5),
    cirq.GridQubit(5, 6),
]



QUBIT_ORDER_14_QUBITS = [
    cirq.GridQubit(2, 4),
    cirq.GridQubit(2, 5),
    cirq.GridQubit(3, 3),
    cirq.GridQubit(3, 4),
    cirq.GridQubit(3, 5),
    cirq.GridQubit(3, 6),
    cirq.GridQubit(4, 3),
    cirq.GridQubit(4, 4),
    cirq.GridQubit(4, 5),
    cirq.GridQubit(4, 6),
    cirq.GridQubit(5, 3),
    cirq.GridQubit(5, 4),
    cirq.GridQubit(5, 5),
    cirq.GridQubit(5, 6),
]

weber_calibration_data = cirq_google.engine.load_median_device_calibration("weber")

readout_errors = {
    # The chance to measure |1> when prepared |0>, isolated for a single qubit
    "single_qubit_p00_error": {},
    
    # The chance to measure |0> when prepared |1>, isolated for a single qubit
    "single_qubit_p11_error": {},
    
    # The chance to measure |1> when prepared |0>, when measuring all device's qubits in parallel
    "parallel_p00_error": {},
    
    # The chance to measure |0> when prepared |1>, when measuring all device's qubits in parallel
    "parallel_p11_error": {}
}

------------

## Calculating average asymmetric readout errrors for the the entire QPU

In [2]:
import json


for readout_error_metric in readout_errors.keys():
    
    aggregated_error = 0
    counter = 0
    
    for error_values in weber_calibration_data[readout_error_metric].values():
        aggregated_error += error_values[0]
        counter += 1
        
    avg_error = aggregated_error / counter
    
    readout_errors[readout_error_metric]["qpu_average"] = avg_error
    
print(json.dumps(readout_errors, indent=4))

{
    "single_qubit_p00_error": {
        "qpu_average": 0.01292830188679246
    },
    "single_qubit_p11_error": {
        "qpu_average": 0.06565283018867925
    },
    "parallel_p00_error": {
        "qpu_average": 0.01583490566037731
    },
    "parallel_p11_error": {
        "qpu_average": 0.07423877358490566
    }
}


-------

## Calculating average asymmetric readout errors for the 12- and 14-qubits schemes

In [3]:
for readout_error_metric in readout_errors.keys():
    
    for scheme in (QUBIT_ORDER_12_QUBITS, QUBIT_ORDER_14_QUBITS):
        
        aggregated_error = 0
        
        for qubit in scheme:
            aggregated_error += weber_calibration_data[readout_error_metric][(qubit, )][0]
        
        num_qubits = len(scheme)
        avg_error = aggregated_error / num_qubits
        readout_errors[readout_error_metric][f"{num_qubits}_scheme_average"] = avg_error

print("Asymmetric readout errors:")
print()
print(json.dumps(readout_errors, indent=4))

Asymmetric readout errors:

{
    "single_qubit_p00_error": {
        "qpu_average": 0.01292830188679246,
        "12_scheme_average": 0.012083333333333333,
        "14_scheme_average": 0.011428571428571429
    },
    "single_qubit_p11_error": {
        "qpu_average": 0.06565283018867925,
        "12_scheme_average": 0.0695,
        "14_scheme_average": 0.0675
    },
    "parallel_p00_error": {
        "qpu_average": 0.01583490566037731,
        "12_scheme_average": 0.015500833333333288,
        "14_scheme_average": 0.014321071428571372
    },
    "parallel_p11_error": {
        "qpu_average": 0.07423877358490566,
        "12_scheme_average": 0.10046458333333337,
        "14_scheme_average": 0.09495107142857144
    }
}


------------

## Calculating average symmetric readout errors

In [4]:
print("Symmetric readout errors:")
print()

for name, metrics_pair in {
    "isolated_readout_errors": ("single_qubit_p00_error", "single_qubit_p11_error"),
    "parallel_readout_errors": ("parallel_p00_error", "parallel_p11_error")
}.items():
    
    print(f"{name}:")

    for key in readout_errors[metrics_pair[0]].keys():
        symmetric_error = (
            readout_errors[metrics_pair[0]][key] + readout_errors[metrics_pair[1]][key]
        ) / 2
    
        print(f"{key}_symmetric = {symmetric_error}")
        
    print()

Symmetric readout errors:

isolated_readout_errors:
qpu_average_symmetric = 0.039290566037735855
12_scheme_average_symmetric = 0.04079166666666667
14_scheme_average_symmetric = 0.039464285714285716

parallel_readout_errors:
qpu_average_symmetric = 0.04503683962264149
12_scheme_average_symmetric = 0.05798270833333333
14_scheme_average_symmetric = 0.05463607142857141



--------------

## Metadata

In [5]:
import sys

print("Python version =", sys.version)
print("cirq version =", cirq.__version__)
print("cirq_google version =", cirq_google.__version__)

Python version = 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
cirq version = 1.4.1
cirq_google version = 1.4.1
